In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('./diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [4]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [6]:
X = scaler.fit_transform(X)

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)

In [48]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout

In [9]:
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=8)) # we are giving these params on our own based on intuition
# so now using keras tuner we automate this process
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

c:\python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
model.fit(X_train, y_train, batch_size=32, epochs=100, validation_data=(X_test, y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step - accuracy: 0.4831 - loss: 0.7330 - val_accuracy: 0.5195 - val_loss: 0.6884
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5842 - loss: 0.6757 - val_accuracy: 0.6234 - val_loss: 0.6387
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7291 - loss: 0.6015 - val_accuracy: 0.6753 - val_loss: 0.6044
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7648 - loss: 0.5665 - val_accuracy: 0.7597 - val_loss: 0.5736
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7672 - loss: 0.5453 - val_accuracy: 0.7597 - val_loss: 0.5538
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7442 - loss: 0.5354 - val_accuracy: 0.7727 - val_loss: 0.5362
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7868 - loss: 0.5028 - val_accuracy: 0.7792 - val_loss: 0.5214
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7743 - loss: 0.4936 - val_accuracy: 0.7792 -

In [11]:
# how to select appropriate optimiser
# no.of nodes in a layer
# no.of hidden layers
# all in one model


In [ ]:
import kerastuner as kt

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_9796\1654478174.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [13]:
def build_model(hp): # build_model is default name hp= hyperparam
    model = Sequential()
    model.add(Dense(32, activation='relu', input_dim=8))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer=hp.Choice('optimizer', values=['adam', 'sgd', 'rmsprop', 'adadelta']), loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [14]:
tuner = kt.RandomSearch(build_model, objective='val_accuracy', max_trials=5)

c:\python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 4 Complete [00h 00m 04s]
val_accuracy: 0.5974025726318359

Best val_accuracy So Far: 0.7662337422370911
Total elapsed time: 00h 00m 28s


In [19]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [20]:
model = tuner.get_best_models(num_models=1)[0]

c:\python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\python312\Lib\site-packages\keras\src\saving\saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [21]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [22]:
model.fit(X_train, y_train, batch_size=32, epochs=100, initial_epoch=6, validation_data=(X_test, y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.7632 - loss: 0.5305 - val_accuracy: 0.7792 - val_loss: 0.5269
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7858 - loss: 0.4965 - val_accuracy: 0.7662 - val_loss: 0.5189
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7706 - loss: 0.5002 - val_accuracy: 0.7597 - val_loss: 0.5146
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7765 - loss: 0.4837 - val_accuracy: 0.7597 - val_loss: 0.5119
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7974 - loss: 0.4647 - val_accuracy: 0.7727 - val_loss: 0.5109
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7569 - loss: 0.5091 - val_accuracy: 0.7662 - val_loss: 0.5085
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7928 - loss: 0.4417 - val_accuracy: 0.7597 - val_loss: 0.5074
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7729 - loss: 0.4602 - val_accuracy: 0.75

In [27]:
def build_model(hp):
    model = Sequential()
    units = hp.Int('units',8,128,step=8) # like range
    model.add(Dense(units=units, activation='relu', input_dim=8))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [28]:
tuner = kt.RandomSearch(build_model, objective='val_accuracy', max_trials=5, directory='myDir')

c:\python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [29]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 05s]
val_accuracy: 0.7792207598686218

Best val_accuracy So Far: 0.7792207598686218
Total elapsed time: 00h 00m 25s


In [30]:
tuner.get_best_hyperparameters()[0].values

{'units': 80}

In [31]:
model = tuner.get_best_models(num_models=1)[0]

c:\python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\python312\Lib\site-packages\keras\src\saving\saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [32]:
model.fit(X_train, y_train, batch_size=32, epochs=100, initial_epoch=6, validation_data=(X_test, y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - accuracy: 0.7618 - loss: 0.4962 - val_accuracy: 0.7727 - val_loss: 0.5176
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7485 - loss: 0.4990 - val_accuracy: 0.7662 - val_loss: 0.5100
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7853 - loss: 0.4502 - val_accuracy: 0.7662 - val_loss: 0.5076
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7890 - loss: 0.4491 - val_accuracy: 0.7532 - val_loss: 0.5060
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7847 - loss: 0.4532 - val_accuracy: 0.7468 - val_loss: 0.5045
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7676 - loss: 0.4723 - val_accuracy: 0.7597 - val_loss: 0.5023
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7492 - loss: 0.4820 - val_accuracy: 0.7662 - val_loss: 0.5014
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7702 - loss: 0.4465 - val_accuracy: 0.7

In [33]:
def build_model(hp):
    model = Sequential()
    model.add(Dense(80, activation='relu', input_dim=8))
    for i in range(hp.Int('num_layers', 1,10)):
        model.add(Dense(80,activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [34]:
tuner = kt.RandomSearch(build_model, objective='val_accuracy', max_trials=3, directory='myDir', project_name='num_layer_proj')

c:\python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [35]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 3 Complete [00h 00m 05s]
val_accuracy: 0.7532467246055603

Best val_accuracy So Far: 0.7792207598686218
Total elapsed time: 00h 00m 19s


In [36]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 3}

In [37]:
model = tuner.get_best_models(num_models=1)[0]

c:\python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\python312\Lib\site-packages\keras\src\saving\saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 12 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [38]:
model.fit(X_train, y_train, batch_size=32, epochs=100, initial_epoch=6, validation_data=(X_test, y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.7634 - loss: 0.4510 - val_accuracy: 0.7532 - val_loss: 0.5112
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7871 - loss: 0.4433 - val_accuracy: 0.7143 - val_loss: 0.5348
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7891 - loss: 0.4466 - val_accuracy: 0.7597 - val_loss: 0.5295
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8073 - loss: 0.4264 - val_accuracy: 0.7597 - val_loss: 0.5201
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8284 - loss: 0.3721 - val_accuracy: 0.7273 - val_loss: 0.5601
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8157 - loss: 0.4083 - val_accuracy: 0.7468 - val_loss: 0.5658
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8224 - loss: 0.3911 - val_accuracy: 0.6948 - val_loss: 0.6062
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8243 - loss: 0.3996 - val_accuracy: 0.70

In [49]:
def build_model(hp):
    model = Sequential()
    counter = 0
    for i in range(hp.Int('num_layers', 1, 10)):
        if counter == 0:
            model.add(Dense(hp.Int('units'+str(i), 8,128,8), 
                      activation=hp.Choice('activation'+str(i), values=['relu', 'tanh', 'sigmoid']),
                      input_dim=8)
                      )
            model.add(Dropout(hp.Choice('Dropout'+str(i), values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
        else:
            model.add(Dense(hp.Int('units'+str(i), 8,128,8), 
                      activation=hp.Choice('activation'+str(i), values=['relu', 'tanh', 'sigmoid']),
                      input_dim=8)
                      )
            model.add(Dropout(hp.Choice('Dropout'+str(i), values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
            
        counter += 1
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=hp.Choice('optimiser', values=['rmsprop', 'adam', 'sgd', 'nadam', 'adadelta']),
                  loss='binary_crossentropy', metrics=['accuracy']
                  )
    return model


In [50]:
tuner = kt.RandomSearch(build_model, objective='val_accuracy', max_trials=3, directory='myDir', project_name='final1')

c:\python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [51]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 3 Complete [00h 00m 05s]
val_accuracy: 0.6428571343421936

Best val_accuracy So Far: 0.7857142686843872
Total elapsed time: 00h 00m 22s


In [52]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 6,
 'units0': 56,
 'activation0': 'tanh',
 'Dropout0': 0.3,
 'optimiser': 'rmsprop',
 'units1': 8,
 'activation1': 'relu',
 'Dropout1': 0.1,
 'units2': 8,
 'activation2': 'relu',
 'Dropout2': 0.1,
 'units3': 8,
 'activation3': 'relu',
 'Dropout3': 0.1,
 'units4': 8,
 'activation4': 'relu',
 'Dropout4': 0.1,
 'units5': 8,
 'activation5': 'relu',
 'Dropout5': 0.1}

In [53]:
model = tuner.get_best_models(num_models=1)[0]


c:\python312\Lib\site-packages\keras\src\saving\saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 16 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [54]:
model.fit(X_train, y_train, epochs=100, initial_epoch=6, validation_data=(X_test, y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - accuracy: 0.7381 - loss: 0.6219 - val_accuracy: 0.7857 - val_loss: 0.5864
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7437 - loss: 0.5901 - val_accuracy: 0.7857 - val_loss: 0.5623
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7448 - loss: 0.5677 - val_accuracy: 0.7727 - val_loss: 0.5388
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7332 - loss: 0.5677 - val_accuracy: 0.7922 - val_loss: 0.5167
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7183 - loss: 0.5483 - val_accuracy: 0.7662 - val_loss: 0.5014
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7381 - loss: 0.5244 - val_accuracy: 0.7662 - val_loss: 0.4954
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7679 - loss: 0.5104 - val_accuracy: 0.7597 - val_loss: 0.4958
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7545 - loss: 0.5046 - val_accuracy: 0.75